In [1]:
import os
import pandas as pd
from PIL import Image
from torchvision import transforms, models

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from memotion_utility import load_data

In [2]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
CSV_FILE = '/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/labels.csv'
ROOT_DIR = '/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/images'
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
LR = 1e-3
epochs = 20
seed = 123
downsample = True

In [3]:
train_df,val_df,test_df = load_data(CSV_FILE,downsample = downsample)

train : 
 label
1    1953
0    1953
Name: count, dtype: int64
val : 
 label
1    343
0    217
Name: count, dtype: int64
test : 
 label
1    856
0    543
Name: count, dtype: int64


In [4]:
train_df

,image_name,text,offensive,label
427,image_5658.png,LET NO ONE TELL YOU YOU ARE UNIMPORTANT,offensive,1
6636,image_5801.png,Facebook was a mistake.” -Mark Zuckerberg,offensive,1
4755,image_3098.jpg,socially amazing penguin's entry MemeCenter,offensive,1
1034,image_5695.jpg,Yo Chuck ima gonna let you finish but.... You ...,offensive,1
6465,image_5216.jpg,I'M JUST GONNA WATCH FINDING NEMO WITH MY SON ...,offensive,1
...,...,...,...,...
5518,image_5858.jpg,FRIENDS WHO GOSSIP ABOUT OTHERS TO YOU ARE GOS...,not offensive,0
3538,image_5036.jpg,SOME WOMEN PLAY MORE MIND GAMES THAN PROFESSOR...,not offensive,0
2156,image_6725.jpg,"GO AHEAD. SAY "" SHUT UP WESLEY!"" YOU BALD- H...",not offensive,0
2732,image_6291.jpg,HEY GIRL GOD CALLED... YOU TO MINISTRY. imgfli...,not offensive,0


In [5]:
class MemeDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.df =  dataframe
        #self.columns_to_map = ['humour', 'sarcasm', 'offensive', 'motivational']
        #self.df[self.columns_to_map] = self.df[self.columns_to_map].applymap(self.map_values)
        self.root_dir = root_dir
        self.transform = transform
        
#     @staticmethod   
#     def map_values(value):
#         if value.lower() in ['funny', 'very_funny', 'hilarious']:
#             return 1
#         elif value.lower() in ['general', 'twisted_meaning', 'very_twisted']:
#             return 1
#         elif value.lower() in ['slight', 'very_offensive', 'hateful_offensive']:
#             return 1
#         elif value.lower() == 'motivational':
#             return 1
#         else:
#             return 0

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.df.iloc[idx]['image_name'])
        image = Image.open(img_name).convert("RGB")
        label = self.df.iloc[idx]['label']

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

In [6]:
# def load_data(csv_file,transform):
#     df = pd.read_csv(csv_file)
#     train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
#     train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)
    
#     train_dataset = MemeDataset(train_df,ROOT_DIR,transform = transform)
#     val_dataset = MemeDataset(val_df,ROOT_DIR,transform = transform)
#     test_dataset = MemeDataset(test_df,ROOT_DIR,transform = transform)
    
#     train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
#     test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     return train_loader,val_loader,test_loader

In [7]:
transform = transforms.Compose([
     transforms.Resize(IMAGE_SIZE),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [8]:
train_dataset = MemeDataset(train_df,ROOT_DIR,transform = transform)
val_dataset = MemeDataset(val_df,ROOT_DIR,transform = transform)
test_dataset = MemeDataset(test_df,ROOT_DIR,transform = transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
train_dataset[0]

(tensor([[[-2.0837, -2.0837, -2.0837,  ..., -1.3130, -1.3130, -1.2959],
          [-2.0837, -2.0837, -2.0837,  ..., -1.3130, -1.3130, -1.2788],
          [-2.0837, -2.0837, -2.0837,  ..., -1.2959, -1.2959, -1.2788],
          ...,
          [-2.1008, -2.1008, -2.1008,  ..., -1.6898, -1.6555, -1.6727],
          [-2.1008, -2.1008, -2.1008,  ..., -1.7069, -1.6555, -1.6898],
          [-2.1008, -2.1008, -2.1008,  ..., -1.7583, -1.7240, -1.7754]],
 
         [[-2.0007, -2.0007, -2.0007,  ..., -1.6331, -1.6331, -1.6331],
          [-2.0007, -2.0007, -2.0007,  ..., -1.6155, -1.6155, -1.6155],
          [-2.0007, -2.0007, -2.0007,  ..., -1.6155, -1.5980, -1.6155],
          ...,
          [-2.0182, -2.0182, -2.0182,  ..., -1.7031, -1.7206, -1.7206],
          [-2.0182, -2.0182, -2.0182,  ..., -1.7206, -1.7206, -1.7381],
          [-2.0182, -2.0182, -2.0182,  ..., -1.7031, -1.6856, -1.7381]],
 
         [[-1.7347, -1.7347, -1.7347,  ..., -1.8044, -1.7870, -1.7522],
          [-1.7347, -1.7347,

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg19 = models.vgg19(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:03<00:00, 170MB/s]


In [11]:
vgg19.classifier[6].in_features

4096

In [12]:
for param in vgg19.parameters():
    param.requires_grad = False

# Modify the classifier part for binary classification
vgg19.classifier[6] = nn.Linear(vgg19.classifier[6].in_features, 2)  # binary classification
vgg19 = vgg19.to(device)

In [13]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(vgg19.classifier[6].parameters(), lr=LR)


for epoch in range(epochs):
    vgg19.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg19(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
    
    # Evaluation
    
    vgg19.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = vgg19(images)
            _, predicted = torch.max(outputs.data, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='macro')
    recall = recall_score(all_labels, all_predictions, average='macro')
    f1 = f1_score(all_labels, all_predictions, average='macro')
    conf_matrix = confusion_matrix(all_labels, all_predictions)
    class_report = classification_report(all_labels, all_predictions, target_names=['Not Offensive', 'Offensive'])

    # Print metrics
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print("Confusion Matrix:")
    print(conf_matrix)
    print("Classification Report:")

  7%|▋         | 9/123 [00:06<01:13,  1.55it/s]/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 123/123 [01:17<00:00,  1.58it/s]


Epoch [1/20], Loss: 0.8113
Accuracy: 46.61%
Precision: 0.53
Recall: 0.52
F1 Score: 0.46
Confusion Matrix:
[[166  51]
 [248  95]]
Classification Report:


  9%|▉         | 11/123 [00:04<00:41,  2.67it/s]/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 123/123 [00:44<00:00,  2.77it/s]


Epoch [2/20], Loss: 0.7881
Accuracy: 44.64%
Precision: 0.50
Recall: 0.50
F1 Score: 0.44
Confusion Matrix:
[[158  59]
 [251  92]]
Classification Report:


 55%|█████▌    | 68/123 [00:24<00:19,  2.78it/s]/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 123/123 [00:44<00:00,  2.77it/s]


...

Epoch [18/20], Loss: 0.7979
Accuracy: 50.18%
Precision: 0.52
Recall: 0.52
F1 Score: 0.50
Confusion Matrix:
[[129  88]
 [191 152]]
Classification Report:


 38%|███▊      | 47/123 [00:17<00:27,  2.74it/s]/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 123/123 [00:44<00:00,  2.75it/s]


Epoch [19/20], Loss: 0.8094
Accuracy: 52.50%
Precision: 0.52
Recall: 0.52
F1 Score: 0.52
Confusion Matrix:
[[112 105]
 [161 182]]
Classification Report:


 53%|█████▎    | 65/123 [00:23<00:22,  2.62it/s]/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 123/123 [00:44<00:00,  2.74it/s]


Epoch [20/20], Loss: 0.8017
Accuracy: 47.14%
Precision: 0.48
Recall: 0.47
F1 Score: 0.47
Confusion Matrix:
[[105 112]
 [184 159]]
Classification Report:


In [14]:
# Test the model and compute metrics
vgg19.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = vgg19(images)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Compute metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='macro')
recall = recall_score(all_labels, all_predictions, average='macro')
f1 = f1_score(all_labels, all_predictions, average='macro')
conf_matrix = confusion_matrix(all_labels, all_predictions)
class_report = classification_report(all_labels, all_predictions, target_names=['Not Offensive', 'Offensive'])

# Print metrics
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Accuracy: 49.68%
Precision: 0.4970
Recall: 0.4969
F1 Score: 0.4905
Confusion Matrix:
[[270 273]
 [431 425]]
Classification Report:
               precision    recall  f1-score   support

Not Offensive       0.39      0.50      0.43       543
    Offensive       0.61      0.50      0.55       856

     accuracy                           0.50      1399
    macro avg       0.50      0.50      0.49      1399
 weighted avg       0.52      0.50      0.50      1399

